# Methods

In [1]:
import numpy as np
from scipy.special import comb, factorial

def eulerian_number(n, m):
    acc = np.float128(0.0)

    for k in range(m + 1):
        c = np.float128(comb(n + 1, k, exact=True))
        acc += (-1.0) ** k * c * (np.float128(m + 1 - k) ** n)

    return acc


def sigmoid(z):
    return 1 / (1 + np.exp(- z))


def sigmoid_derivative(z, n=2):
    sigma_z = sigmoid(z)

    acc = np.float128(0.0)

    for k in range(1, n + 1):
        acc += ((-1) ** (k - 1)) * eulerian_number(n, k - 1) * (sigma_z ** k) * ((1 - sigma_z) ** (n + 1 - k))

    return acc


# Load RBM

In [2]:
import torch
import rbm_pytorch

In [3]:
CHECKPOINT = "models/L8/1.8/trained_rbm.pytorch.last"

In [4]:
rbm = rbm_pytorch.RBM(64, 64)

rbm_pytorch.py:106: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  nn.init.normal(self.W,mean=0, std=0.01)


In [5]:
rbm.load_state_dict(torch.load(CHECKPOINT))

# Compute bounds

We can write the (n + 1)-th cumulant as

\begin{equation*}
\kappa_{\mu}^{(n + 1)} = \sum_{k=1}^{n} \big( -1 \big)^{k - 1} A_{n, k-1} \sigma (b_{\mu})^k \big( 1 - \sigma(b_{\mu}) \big)^{n + 1 - k}
\end{equation*}

Or equivalently, by shifting \\(k\\) by \\(-1\\),

\begin{equation*}
\kappa_{\mu}^{(n + 1)} = \sum_{k = 0}^{n - 1} \big( -1 \big)^k A_{n, k} \sigma(b_{\mu})^{k + 1} \big( 1 - \sigma(b_{\mu}) \big)^{n - k}
\end{equation*}

Consider the case where \\(b_{\mu} \in [a, b]\\) for some *bounded* non-degenerate interval with \\(a, b \in \mathbb{R}\\), and let

\begin{equation*}
\alpha = \max \left\{ \max_{b_{\mu} \in [a, b]} \sigma(b_{\mu}), \max_{b_{\mu} \in [a, b]} \big( 1 - \sigma(b_{\mu}) \big) \right\}
\end{equation*}

we observe

\begin{equation*}
\begin{split}
  \left| \kappa_{\mu}^{(n + 1)} \right| &= \left| \sum_{k=0}^{n - 1} \big( - 1 \big)^k A_{n, k} \sigma(b_{\mu})^{k + 1} \big( 1 - \sigma(b_{\mu} \big)^{n - k} \right| \\
  &\le \sum_{k=0}^{n - 1} \left| A_{n, k} \right| \left| \sigma(b_{\mu})^{k + 1} \big( 1 - \sigma(b_{\mu}) \big)^{n - k} \right| \\
  & \le \sum_{k=0}^{n - 1} \left| A_{n, k} \right| \alpha^{k + 1} \alpha^{n - k} \\
  &= \alpha^{n + 1} \sum_{k=0}^{n - 1} A_{n, k} \\
  &= \alpha^{n + 1} n!
\end{split}
\end{equation*}

where we've used the fact that

\begin{equation*}
A_{n, k} \ge 0, \quad \forall n \in \mathbb{N} \quad \text{and} \quad \sum_{k=0}^{n - 1} A_{n, k} = n!, \quad n \ge 1
\end{equation*}

Relating to the coefficient of the Taylor expansion for the energy, we instead consider \\(\kappa_{\mu}^{(n + 1)} / (n + 1)!\\):

\begin{equation*}
\frac{\left| \kappa_{\mu}^{(n + 1)} \right|}{(n + 1)!} \le \frac{\alpha^{n + 1}}{n + 1}
\end{equation*}

Or wrt. $n$:

\begin{equation*}
\frac{\left| \kappa_{\mu}^{(n)} \right|}{n!} \le \frac{\alpha^{n}}{n}
\end{equation*}

In [6]:
z = np.float128(rbm.h_bias.detach().numpy())

Compute

$$
\frac{\left| \kappa^{(n)} \right|}{n!} = \frac{\left| \sigma^{(n - 1)} \right|}{n!}
$$

and upper-bound 

$$
\frac{\alpha^{n}}{n}
$$

In [7]:
n = 50

In [8]:
# compute the abs of \kappa^{(n)}
kappa = np.abs(sigmoid_derivative(z, n=n - 1)) / np.float128(factorial(n), exact=True)

In [9]:
# compute upper-bounds
y = sigmoid(z)
alpha = np.max([y, 1 - y], axis=0)
ub = np.abs((alpha ** (n)) / (n))

In [10]:
# verify bounds
kappa <= ub

array([False,  True, False,  True,  True, False, False,  True, False,
       False, False,  True, False, False,  True, False, False,  True,
        True, False, False,  True, False,  True, False, False,  True,
       False, False,  True, False, False, False, False,  True,  True,
       False, False, False, False, False, False,  True, False, False,
        True, False, False, False,  True, False,  True, False, False,
        True, False, False, False, False, False,  True, False,  True,
       False])

In [11]:
kappa

array([1.47411269e+01, 8.00013576e-37, 2.11172572e-01, 2.00558305e-40,
       5.22905477e-39, 1.01627746e+02, 7.05141101e-01, 3.25076514e-31,
       6.42562153e+00, 2.32928780e+01, 1.48016158e+00, 4.63129047e-32,
       3.78370904e-08, 2.48921654e-01, 1.63135706e-32, 3.27811472e-01,
       9.94352364e-04, 7.49396113e-28, 3.52681264e-38, 1.05870450e+02,
       2.20888956e-01, 1.91122507e-38, 1.58637834e+01, 2.37127774e-58,
       4.84326330e+01, 9.07043328e+01, 1.26372406e-32, 8.87759617e-13,
       1.82712641e-07, 1.07597187e-55, 3.36766589e+00, 7.39657695e+00,
       1.19318481e-01, 7.98601565e-02, 1.81853271e-57, 1.38513130e-30,
       1.93997107e+00, 1.44400388e-01, 4.79998420e+01, 1.01991250e+02,
       1.02401427e+02, 2.08776186e-01, 6.94363435e-36, 2.39220229e+00,
       3.46426469e+01, 1.67430587e-46, 1.05090658e+02, 9.70118018e-01,
       1.08598038e-01, 3.38522311e-24, 4.73144652e-14, 6.43596323e-30,
       1.31389227e-01, 1.18488416e+00, 7.21680193e-49, 1.18331424e+00,
      

In [12]:
ub

array([1.89520089e-02, 4.82800427e-03, 1.99853762e-02, 1.24378656e-02,
       1.11480463e-02, 9.77941812e-03, 1.99511273e-02, 2.01856594e-06,
       1.95501626e-02, 4.00404216e-04, 1.32814807e-05, 7.11865033e-05,
       8.77493903e-14, 1.99827609e-02, 4.16384015e-04, 1.99772943e-02,
       4.03953300e-09, 2.38009446e-08, 8.18216608e-03, 6.21385609e-03,
       1.99847030e-02, 8.28244176e-03, 1.88697554e-02, 1.99974778e-02,
       1.14832105e-03, 1.17418105e-02, 3.58731445e-04, 1.86610370e-16,
       4.52890533e-13, 1.99848327e-02, 1.97655121e-02, 1.94812849e-02,
       1.99917384e-02, 1.99944709e-02, 1.99958583e-02, 2.76245972e-05,
       1.98652562e-02, 1.99900013e-02, 1.63211333e-02, 9.69178054e-03,
       9.58940816e-03, 1.99855422e-02, 4.34882879e-03, 1.98337155e-02,
       1.74319596e-02, 1.90924750e-02, 5.92269153e-03, 1.99327314e-02,
       1.99924808e-02, 7.81760021e-10, 2.40795992e-15, 6.46159141e-06,
       1.99909024e-02, 1.99178090e-02, 1.96710524e-02, 1.99179181e-02,
      

Since we observe that some values fail the upper-bound check, it might be useful to check if the following is verified for such large $n$:

$$
\sum_{m=0}^{n - 1} A_{n, m} = n!
$$

In [13]:
np.float128(factorial(n, exact=True))

3.0414093201713375576e+64

In [14]:
np.sum([eulerian_number(n, m) for m in range(n)])

2.5192173461513654015e+70

As we can see, there is a huge discrepancy between the two computations, implying that numerical error plays an important role in why we cannot get the series to converge numerically.